In [1]:
import requests
import zipfile
import io
import os
import pandas as pd
from datetime import datetime, timedelta
import time

Binance API doc
https://developers.binance.com/docs/binance-spot-api-docs/testnet/rest-api/market-data-endpoints#klinecandlestick-data

In [32]:
from dotenv import load_dotenv
from dune_client.client import DuneClient

In [33]:
def get_dune_client():
    dotenv_path = "/Users/zokum/local-Workspace/uf/uniswap-tvl-and-volume-breakdown-versions-and-chains/.env"
    load_dotenv(dotenv_path)
    return DuneClient.from_env()

In [34]:
dune = get_dune_client()

In [43]:
# Parameters
symbol = "ETHUSDT" 
interval = "1s"
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 4, 30)
output_folder = "./price_1s"
os.makedirs(output_folder, exist_ok=True)

In [44]:
# Define the header based on Binance documentation
columns = [
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
]

In [45]:
all_data = []

In [46]:
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    filename = f"{symbol}-{interval}-{date_str}.zip"
    url = f"https://data.binance.vision/data/spot/daily/klines/{symbol}/{interval}/{filename}"
    try:
        res = requests.get(url)
        res.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(res.content)) as z:
            extracted_file = z.namelist()[0]
            with z.open(extracted_file) as f:
                lines = f.read().decode("utf-8").splitlines()
                daily_data = [line.split(",") for line in lines]
                all_data.extend(daily_data)
        print(f"✅ Downloaded {date_str}")
    except Exception as e:
        print(f"❌ Failed on {date_str}: {e}")
    current_date += timedelta(days=1)
    
    time.sleep(0.1) 

✅ Downloaded 2025-01-01
✅ Downloaded 2025-01-02
✅ Downloaded 2025-01-03
✅ Downloaded 2025-01-04
✅ Downloaded 2025-01-05
✅ Downloaded 2025-01-06
✅ Downloaded 2025-01-07
✅ Downloaded 2025-01-08
✅ Downloaded 2025-01-09
✅ Downloaded 2025-01-10
✅ Downloaded 2025-01-11
✅ Downloaded 2025-01-12
✅ Downloaded 2025-01-13
✅ Downloaded 2025-01-14
✅ Downloaded 2025-01-15
✅ Downloaded 2025-01-16
✅ Downloaded 2025-01-17
✅ Downloaded 2025-01-18
✅ Downloaded 2025-01-19
✅ Downloaded 2025-01-20
✅ Downloaded 2025-01-21
✅ Downloaded 2025-01-22
✅ Downloaded 2025-01-23
✅ Downloaded 2025-01-24
✅ Downloaded 2025-01-25
✅ Downloaded 2025-01-26
✅ Downloaded 2025-01-27
✅ Downloaded 2025-01-28
✅ Downloaded 2025-01-29
✅ Downloaded 2025-01-30
✅ Downloaded 2025-01-31
✅ Downloaded 2025-02-01
✅ Downloaded 2025-02-02
✅ Downloaded 2025-02-03
✅ Downloaded 2025-02-04
✅ Downloaded 2025-02-05
✅ Downloaded 2025-02-06
✅ Downloaded 2025-02-07
✅ Downloaded 2025-02-08
✅ Downloaded 2025-02-09
✅ Downloaded 2025-02-10
✅ Downloaded 202

In [47]:
# Create dataframe
df = pd.DataFrame(all_data, columns=columns)

# Convert microsecond timestamps to datetime
df["open_time"] = pd.to_datetime(df["open_time"].astype("int64") // 1_000_000, unit="s")
df["close_time"] = pd.to_datetime(df["close_time"].astype("int64") // 1_000_000, unit="s")

In [48]:
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_volume,taker_buy_quote_volume,ignore
0,2025-01-01 00:00:00,3337.78000000,3337.78000000,3337.78000000,3337.78000000,0.27470000,2025-01-01 00:00:00,916.88816600,4,0.00000000,0.00000000,0
1,2025-01-01 00:00:01,3337.78000000,3337.79000000,3337.78000000,3337.78000000,0.51770000,2025-01-01 00:00:01,1727.96873500,6,0.00290000,9.67959100,0
2,2025-01-01 00:00:02,3337.79000000,3337.79000000,3337.78000000,3337.78000000,0.01500000,2025-01-01 00:00:02,50.06680000,2,0.01000000,33.37790000,0
3,2025-01-01 00:00:03,3337.78000000,3337.78000000,3337.41000000,3337.41000000,0.97980000,2025-01-01 00:00:03,3270.18997700,25,0.10880000,363.11129600,0
4,2025-01-01 00:00:04,3337.42000000,3337.42000000,3337.42000000,3337.42000000,0.00750000,2025-01-01 00:00:04,25.03065000,2,0.00750000,25.03065000,0


In [49]:
df.shape

(10166379, 12)

In [50]:
# Truncate columns if desired
df_mod = df[["close_time", "close"]]

In [51]:
df_mod['close'] = pd.to_numeric(df_mod['close'], errors='coerce')

/var/folders/d7/px4831yx38d7k85rs8_2n93r0000gn/T/ipykernel_2564/3874359909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mod['close'] = pd.to_numeric(df_mod['close'], errors='coerce')


In [52]:
print(df_mod.dtypes)

close_time    datetime64[ns]
close                float64
dtype: object


In [53]:
df_mod

,close_time,close
0,2025-01-01 00:00:00,3337.78
1,2025-01-01 00:00:01,3337.78
2,2025-01-01 00:00:02,3337.78
3,2025-01-01 00:00:03,3337.41
4,2025-01-01 00:00:04,3337.42
...,...,...
10166374,2025-04-29 23:59:55,1798.38
10166375,2025-04-29 23:59:56,1798.07
10166376,2025-04-29 23:59:57,1797.88
10166377,2025-04-29 23:59:58,1797.81


In [54]:
# # Save to CSV
# df_mod.to_csv(os.path.join(output_folder, "ethusdc_1s_jan1_apr29.csv"), index=False)
# print("✅ CSV saved and ready for upload!")

In [55]:
# df_mod_csv = df_mod.to_csv(index=False)

In [56]:
dune.create_table(
        namespace="uniswap_fnd",
        table_name="binance_ethusdt_1s_1jan2025_29apr2025",
        description="Binance ETH USDT 1s price feed",
        schema= [
            {"name": "close_time", "type": "timestamp"},
            {"name": "close", "type": "double"}
        ],
        is_private=False
)

CreateTableResult(example_query='select * from dune.uniswap_fnd.binance_ethusdt_1s_1jan2025_29apr2025 limit 10', full_name='dune.uniswap_fnd.binance_ethusdt_1s_1jan2025_29apr2025', namespace='uniswap_fnd', table_name='binance_ethusdt_1s_1jan2025_29apr2025')

In [57]:
chunk_size = 2_000_000
for i in range(0, len(df_mod), chunk_size):
    chunk = df_mod.iloc[i:i + chunk_size]
    chunk_csv = chunk.to_csv(index=False)
    print(f"Inserting rows {i} to {i + len(chunk)}")
    dune.insert_table(
        namespace="uniswap_fnd",
        table_name="binance_ethusdt_1s_1jan2025_29apr2025",
        data=chunk_csv,
        content_type="text/csv"
    )

Inserting rows 0 to 2000000
Inserting rows 2000000 to 4000000
Inserting rows 4000000 to 6000000
Inserting rows 6000000 to 8000000
Inserting rows 8000000 to 10000000
Inserting rows 10000000 to 10166379


In [ ]:
# # Upload to Dune -- file less than 200MB
# upload_success = dune.upload_csv(
#     table_name="binance_ethusdc_1s_1jan2025_29apr2025",
#     data=df_mod_csv,
#     is_private=True
# )
# print("Upload status:", upload_success)